In [1]:
import ipywidgets as ipw
from IPython.display import display, clear_output
from jinja2 import Template
import io
from aiidalab_widgets_base import ComputationalResourcesWidget, viewer, ProcessMonitor
import traitlets
from aiida.orm import Node, ProcessNode

In [2]:
inp_template = Template(""" &input
   zed={{ zed }},
   rel={{ rel }},
   config='{{ config }}',
   iswitch={{ iswitch }},
   dft='{{ dft }}'
 /
 &inputp
   lpaw={{ lpaw }},
   pseudotype={{ pseudotype }},
   file_pseudopw='{{ file_pseudopw }}',
   author='{{ author }}',
   lloc=-1,
   rcloc={{ rcloc }}
   which_augfun='{{ which_angfun }}',
   rmatch_augfun_nc={{ rmatch_augfun_nc }},
   tm={{ tm }}
 /
{{ card }}
""")

_CARD_DEFAULT = """4
1S  1  0  2.00  0.00  0.80  1.00  0.0
2S  2  0  1.00  0.00  0.80  1.00  0.0
2P  2  1  0.00  0.00  0.85  1.20  0.0
2P  2  1  0.00  1.00  0.85  1.20  0.0
"""

_ZED_DEFAULD = 3


In [3]:
from aiidalab_qe.widgets import CalcJobOutputFollower, LogOutputWidget
from aiidalab_widgets_base import register_viewer_widget

@register_viewer_widget("process.calculation.calcjob.CalcJobNode.")
class CalcJobNodeViewerWidget(ipw.VBox):
    def __init__(self, calcjob, **kwargs):
        self.calcjob = calcjob
        self.output_follower = CalcJobOutputFollower()
        self.log_output = LogOutputWidget()

        self.output_follower.observe(self._observe_output_follower_lineno, ["lineno"])
        self.output_follower.calcjob = self.calcjob

        super().__init__(
            [ipw.HTML(f"CalcJob: {self.calcjob}"), self.log_output], **kwargs
        )

    def _observe_output_follower_lineno(self, _):
        with self.hold_trait_notifications():
            self.log_output.filename = self.output_follower.filename
            self.log_output.value = "\n".join(self.output_follower.output)

class NodeViewWidget(ipw.VBox):

    node = traitlets.Instance(Node, allow_none=True)

    def __init__(self, **kwargs):
        self._output = ipw.Output()
        super().__init__(children=[self._output], **kwargs)

    @traitlets.observe("node")
    def _observe_node(self, change):
        if change["new"] != change["old"]:
            with self._output:
                clear_output()
                if change["new"]:
                    display(viewer(change["new"]))
        

In [4]:
class Ld1PseudoGeneratorWidget(ipw.VBox):

    process = traitlets.Instance(ProcessNode, allow_none=True)

    def __init__(self):

        ## input
        # zed: Int Txt
        self.zed = ipw.BoundedIntText(
            value=_ZED_DEFAULD,
            min=1,
            max=90,
            step=1,
            description='zed:',
            disabled=False
        )

        # nrel exclusive Toggle
        self.rel = ipw.ToggleButtons(
            options=['Non', 'Scalar', 'Full'],
            description='Relativistic:',
            disabled=False,
            tooltips=['Non-relativistic', 'Scalar-relativistic', 'Full-relativistic'],
        )

        # config set to default not setable
        self.config = ipw.Text(
            value='[He] 2s1 2p0',
            placeholder='Electrons configuration.',
            description='config:',
            disabled=False
        )

        # iswitch=3 fixed
        self.iswitch_value = 3

        # dft: exclusive toggle 'PBE' as default
        self.dft = ipw.ToggleButtons(
            options=['PBE', 'PBESOL', 'PZ'],
            description='dft_exc:'
        )

        ## inputp
        # lpaw: 
        self.lpaw_value = '.false.'

        # pseudotype: NC->1, PAW/US ->3
        self.pseudotype = ipw.ToggleButtons(
            options=['PAW/US', 'NC'],
            description='pseudotype:',
            disabled=False,
        )

        # file_pseudopw. text box. check must end with upf
        self.file_pseudopw = ipw.Text(
            value='pseudo.upf',
            placeholder='Output upf pseudo file name',
            description='filename:',
            disabled=False
        )

        # author: text box
        self.author = ipw.Text(
            value='anonymous',
            description='author',
            disabled=False,
        )

        # rcloc: float text
        self.rcloc = ipw.FloatText(
            value=0.6,
            description='rcloc:',
            disabled=False,
        )

        # fixed: which_augfun='PSQ' and rmatch_augfun_nc=.true.
        self.which_augfun_value = 'PSQ'
        self.rmatch_augfun_nc_value = True

        # tm: toggle TM or RRKJ: TM -> true RRKJ -> false
        self.pseudization = ipw.ToggleButtons(
            options=['TM', 'RRKJ'],
            description='pseudization:'
        )

        # PseudoPotentialGenerationCards
        # big text box
        self.card = ipw.Textarea(
            value=_CARD_DEFAULT,
            placeholder='Pseudo potential generation card',
            description='Cards:',
            disabled=False,
            layout=ipw.Layout(height="200px", width="auto"),
        )

        # Generate button
        self.generate_inp = ipw.Button(
            description='Generate inp',
            disabled=False,
        )
        self.generate_inp.on_click(self._on_generate_inp_click)

        self.output_inp = ipw.Output()

        self.generate_pseudo = ipw.Button(
            description='Generate pseudo',
            disabled=False,
        )
        self.generate_pseudo.on_click(self._on_generate_pseudo_click)

        self.code = ComputationalResourcesWidget(    
            value='ld1@localhost',
            description="ld1.x:", input_plugin="sssp.pseudo.ld1"
        )

        self.inp = ''
        self.node_view = NodeViewWidget(layout={"width": "auto", "height": "auto"})

        # pseudo_output to show and download
        self.pseudo_output = LogOutputWidget()
        self.pseudo_verify = ipw.HTML('www.nnn')
        self.process_monitor = ProcessMonitor(
            timeout=0.8,
            callbacks=[
                self._pseudo_output_update,
            ],
        )
        ipw.dlink((self, "process"), (self.process_monitor, "process"))

        super().__init__(
            children=[
                self.code,
                self.zed,
                self.rel,
                self.dft, 
                self.config,
                self.pseudotype, 
                self.file_pseudopw,
                self.author,
                self.rcloc, 
                self.pseudization, 
                self.card,
                ipw.HBox(children=[self.generate_inp, self.generate_pseudo]),
                self.output_inp,
                self.node_view,
                self.pseudo_output,
                self.pseudo_verify,
            ]
        )

    def _pseudo_output_update(self):
        try:
            self.pseudo_output.filename = self.process.inputs.filename.value
            self.pseudo_output.value = self.process.outputs.output_pseudo.get_content()
        except:
            self.pseudo_output.filename = '[Not avaliable]'
            self.pseudo_output.value = f'{self.process.process_state.value}'

    def _on_generate_pseudo_click(self, _):
        """submit a ld1 pseudo aiida job"""
        from aiida import orm
        from aiida.engine import submit
        from aiida.plugins import CalculationFactory

        Ld1Calculation = CalculationFactory('sssp.pseudo.ld1')
        # string -> SinglefileData
        inputs = {
            'code': self.code.value,
            'filename': orm.Str(self.file_pseudopw.value),
            'parameters': orm.SinglefileData(file=io.BytesIO(self.inp.encode('utf-8')))
        }
        # submit
        self.process = submit(Ld1Calculation, **inputs)
        self.node_view.node  = self.process

    def _on_generate_inp_click(self, _):
        self.inp = self._generate_inp_content()

        self.output_inp.clear_output()
        with self.output_inp:
            print(self.inp)

    def _generate_inp_content(self) -> str:
        zed = self.zed.value
        
        if self.rel.value == 'Non':
            rel = 0
        elif self.rel.value == 'Scalar':
            rel = 1
        else:
            rel = 2

        config = self.config.value
        iswitch = self.iswitch_value
        
        dft = self.dft.value

        lpaw = self.lpaw_value

        if self.pseudotype.value == 'NC':
            pseudotype = 1
        else:
            pseudotype = 3

        author = self.author.value
        rcloc = self.rcloc.value
        which_augfun = self.which_augfun_value
        if self.rmatch_augfun_nc_value:
            rmatch_augfun_nc = '.true.'
        else:
            rmatch_augfun_nc = '.false.'
        file_pseudopw = self.file_pseudopw.value

        if self.pseudization.value == 'TM':
            tm = '.true.'
        else:
            tm = '.false.'

        card = self.card.value

        inp_kwargs = {
            'zed': zed, 'rel': rel, 'config': config, 'iswitch': iswitch,
            'dft': dft, 'lpaw': lpaw, 'pseudotype': pseudotype,
            'author': author, 'rcloc': rcloc, 'which_angfun': which_augfun,
            'file_pseudopw': file_pseudopw,
            'rmatch_augfun_nc': rmatch_augfun_nc, 'tm': tm, 'card': card,
        }
        inp = inp_template.render(**inp_kwargs)

        return inp

In [5]:
pseudo_generator = Ld1PseudoGeneratorWidget()

display(pseudo_generator)

/opt/conda/lib/python3.7/site-packages/aiida/transports/transport.py:201: AiidaDeprecationWarning: method is deprecated, use `aiida.plugins.entry_point.get_entry_point_names` instead
  warnings.warn(message, AiidaDeprecationWarning)  # pylint: disable=no-member
/opt/conda/lib/python3.7/site-packages/aiida/schedulers/scheduler.py:93: AiidaDeprecationWarning: method is deprecated, use `aiida.plugins.entry_point.get_entry_point_names` instead
  warnings.warn(message, AiidaDeprecationWarning)  # pylint: disable=no-member


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ld1PseudoGeneratorWidget(children=(ComputationalResourcesWidget(children=(HBox(children=(Dropdown(description=…